In [1]:
import pandas as pd
import yfinance as yf
import os
import random
import numpy as np

# df = pd.read_csv("all_history.csv")
# df = df.set_index("Date")
# df.head(5)

In [8]:
df = pd.read_csv("all_history.csv")
df = df.reset_index()
df.head(5)

,index,Unnamed: 0,Date,SPY,A,AA,AAAP,AABB,AAC,AAL,...,ZSTN,ZTR,ZTS,ZULU,ZUMZ,ZVLO,ZVTK,ZWBC,ZYNE,ZYXI
0,0,0,01/03/2000,94.262558,43.76,70.69,24.50,77.49,9.87,18.19,...,7.52,2.46,29.05,0.06,12.44,26.75,3750000.0,12.75,16.25,1.28
1,1,1,01/04/2000,90.576347,40.42,71.02,25.00,77.49,9.96,19.33,...,7.20,2.48,29.06,0.06,12.26,25.00,10000000.0,12.75,19.32,1.31
2,2,2,01/05/2000,90.738327,37.91,75.11,25.26,77.49,9.78,19.05,...,7.50,2.49,29.08,0.05,12.05,27.50,7500000.0,30.00,24.54,1.32
3,3,3,01/06/2000,89.280045,36.46,74.13,25.02,77.49,9.83,19.81,...,7.55,2.51,29.07,0.05,12.12,25.50,4450000.0,25.00,35.14,1.31
4,4,4,01/07/2000,94.465111,39.50,73.91,24.62,77.49,9.90,20.27,...,7.31,2.56,29.98,0.05,11.75,27.34,2750000.0,16.00,29.73,1.29


In [28]:
etf_df = pd.read_csv("all_history_etf.csv")
etf_df = etf_df.drop(etf_df.columns[[0]], axis=1)
etf_df.head()

,Date,-ZSILEU.SW,-ZSILHE.SW,0050.TW,0051.TW,0052.TW,0053.TW,0054.TW,0055.TW,0056.TW,...,ZWB.TO,ZWE.TO,ZWH-U.TO,ZWH.TO,ZWU.TO,ZXM-B.TO,ZXM.TO,ZYAU.AX,ZYUS.AX,^FTCS
0,01/03/2000,83.44,155.43,43.62,23.44,27.38,18.60,23.28,9.02,14.17,...,8.17,13.40,17.65,9.69,8.07,18.38,18.22,7.90,7.89,3717.98
1,01/04/2000,85.05,151.75,42.80,23.02,26.67,18.15,22.84,8.92,13.91,...,8.19,13.34,17.39,9.64,8.04,18.38,18.22,8.01,7.95,3555.62
2,01/05/2000,84.94,145.99,42.89,23.15,26.35,17.94,22.71,9.26,14.07,...,8.21,13.34,17.56,9.76,8.03,18.38,18.22,8.01,7.88,3373.04
3,01/06/2000,86.70,152.56,40.99,22.10,24.66,16.85,21.66,9.03,13.70,...,8.23,13.56,17.49,9.83,8.02,18.38,18.22,7.98,7.90,3325.04
4,01/07/2000,88.55,151.81,41.42,22.20,24.43,16.70,21.83,9.33,14.00,...,8.25,13.55,17.61,9.82,8.06,18.38,18.22,7.93,7.86,3310.51


In [30]:
etf_df.loc[1:3, ["Date", "SPY"]]

,Date,SPY
1,01/04/2000,93.69
2,01/05/2000,93.86
3,01/06/2000,92.35


In [ ]:
# def get_history(ticker, interval="1d", period="1y",
#                 start_date="2000-01-01", end_date="2023-11-14"
#                 ):
#     # Interval can be "1d", "5d", "1wk", "1mo", or "3mo"
#     # start and end are given in str, dt, or int: "YYYY-MM-DD", datetime, or epoch respectively.
#     # Alternatively, we can use a period argument in place of start and end dates.
#     # period can equal "3mo", "6mo", "1y", "2y", "5y", "10y", "ytd", or "max".
    
#     # TODO: Validate based on duration and/or interval to prevent out-of-bounds dates
    
#     stock = yf.Ticker(ticker)
#     history = stock.history(interval=interval, period=period, start=start_date, end=end_date)
    
#     # if len(history) == 0:
#     #     print("No history available.")
#     #     return
    
#     # Reset index to make Date accessible
#     history = history.reset_index()
#     # print(history)
    
#     # Return the dataframe
#     return history

# benchmark_history_df = get_history("SPY")
# benchmark_history_df = benchmark_history_df[["Date", "Close"]]
# benchmark_history_df['Date'] = benchmark_history_df['Date'].dt.strftime('%m/%d/%Y')
# benchmark_history_df.set_index("Date", inplace=True)
# benchmark_history_df.rename(columns={"Close": "SPY"}, inplace=True)
# print(benchmark_history_df.columns)

# large_df = pd.read_csv(os.path.join(project_dir, "all_history.csv"))
# # print(large_df.columns[-1])

# new_df = pd.merge(benchmark_history_df, large_df, on="Date", how="outer")
# # print(new_df.columns[-1])
# new_df.head(10)

# print(new_df["AA"].head(10))


In [30]:
# new_df = new_df.reset_index()
# new_df.loc[new_df.index[-1], "SPY"]
# new_df.loc[new_df.index[-1], "Date"]
# new_df.loc[:]
# new_df.to_csv(path_or_buf=os.path.join(project_dir, "all_history.csv"))

In [2]:
reset_df = df.reset_index()
# date_value = reset_df.loc[reset_df["Date"] == "01/05/2000", "Date"].index[0]
reset_df.head()


,Date,Unnamed: 0,SPY,A,AA,AAAP,AABB,AAC,AAL,AAMC,...,ZSTN,ZTR,ZTS,ZULU,ZUMZ,ZVLO,ZVTK,ZWBC,ZYNE,ZYXI
0,01/03/2000,0,94.262558,43.76,70.69,24.50,77.49,9.87,18.19,9.26,...,7.52,2.46,29.05,0.06,12.44,26.75,3750000.0,12.75,16.25,1.28
1,01/04/2000,1,90.576347,40.42,71.02,25.00,77.49,9.96,19.33,8.65,...,7.20,2.48,29.06,0.06,12.26,25.00,10000000.0,12.75,19.32,1.31
2,01/05/2000,2,90.738327,37.91,75.11,25.26,77.49,9.78,19.05,8.26,...,7.50,2.49,29.08,0.05,12.05,27.50,7500000.0,30.00,24.54,1.32
3,01/06/2000,3,89.280045,36.46,74.13,25.02,77.49,9.83,19.81,8.39,...,7.55,2.51,29.07,0.05,12.12,25.50,4450000.0,25.00,35.14,1.31
4,01/07/2000,4,94.465111,39.50,73.91,24.62,77.49,9.90,20.27,8.96,...,7.31,2.56,29.98,0.05,11.75,27.34,2750000.0,16.00,29.73,1.29


In [3]:
# We will store the period, start date, and portfolios in a class file.  We can then have
# an array of Portfolios to compare.

class Portfolio:
    # instance variables/attributes...
    size = 10 # The initial size or num_stocks in the portfolio
    n_periods = 10 # The number of trading days the portfolio analysis takes place over
    start_date_index = 0
    
    weights = pd.DataFrame({}) # Weights that will be assigned to each of the assets in the portfolio
    
    stdev = 0
    hpr = 0
    hpr_annualized = 0
    risk_adj_return = 0
    alpha = 0
    
    history_df = pd.DataFrame({})
    analysis_df = pd.DataFrame({})
    
    # Constructor
    def __init__(self, size, n_periods, start_date="Random"):
        
        self.size = size
        self.n_periods = n_periods
        
        # If a number is given for start_date, set start_date_index to that number
        if (type(start_date) == int):
            # print("Chose an int index for start_date_index")
            self.start_date_index = start_date
        
        # Else if a date is given, find that index
        elif (start_date != "Random"):
            # TODO: Validate format and catch errors...
            # Should:
            #     - Catch imporperly-formatted dates, i.e. 10-20-2020 and convert to 10/20/2020
            #     - Catch missing dates such as Saturdays or holidays, and populate the start to the next business day
            
            start_date_index = reset_df.loc[reset_df["Date"] == start_date, "Date"].index[0]
            # print("Start date: "+)
            # print("Set start date index for portfolio instance to: "+str(start_date_index))
            # print(reset_df.loc[start_date_index, "Date"])
            # print("Found index within df at "+str(start_date))
            
            self.start_date_index = start_date_index
            
        
        # Else (or if no args are given) - randomize
        else:
            self.randomize_start()
        
        
        # Generate history_df
        self.create_portfolio()
        
        # Assign weights
        self.assignWeights()
        
        print("Initialized new Portfolio instance")
        
    
    # Methods...
    
    # NOTE: Both randomize_ricker and randomize_start need access to the massive CSV file and currently take a pre-made df
    # not as an argument, but as a variable inside them.  This will need to be changed in whatever file I move this to.
    def randomize_ticker(self):
        ticker_index = random.randint(0, len(df.columns) - 1)
        sub_df = df.iloc[:, ticker_index]
        return sub_df.name.upper()
    

    def randomize_start(self):
        """Takes a length parameter that will be added to a random start date and represent the period analyzed"""
        # NOTE: This uses the length of df, which does not exist in the class.  Might be better to define the read_csv here in the class file.
        # Perhaps pass it as an argument in randomize_start and have that param point to the read_csv
        rand_start = random.randint(0, (len(df) - self.n_periods))
        
        self.start_date_index = rand_start
        return self.start_date_index
    
    
    def create_portfolio(self):
    #   df to hold the data from each ticker
        prices_df = pd.DataFrame({})

        start_index = self.start_date_index
        
        i = 0
        while i < self.size:
    #       Run the randomizer and trim values
            ticker = self.randomize_ticker()
            ticker_index = df.columns.get_loc(ticker)
            ticker_df = df.iloc[start_index : start_index + self.n_periods, [ticker_index]]

    #       Clean null values and randomize again if necessary
            ticker_df = ticker_df.dropna()
            if (len(ticker_df) < self.n_periods):
                # Break and decrement i if there are null values in the date range
                # print("\nSeries contains null values.  Choosing a different ticker...")
                i -= 1

            else:
                # Add the data as intended.
                if prices_df.empty:
                    prices_df = ticker_df
                else:
                    prices_df = pd.concat([prices_df, ticker_df], axis=1)
    #                   prices_df = prices_df.merge(prices_df, ticker_df)
    #       
            i += 1
        
        # Load with the percentage change columns
        for item in prices_df.columns:
    #       First, load with values
            prices_df[str(item)+" % chg"] = prices_df[item].pct_change()
        
        # print("\n\nFinal prices_df: ")
        # print(prices_df.head(3))
        
    
        self.history_df = prices_df
        return self.history_df
    
    
#   Assign weights
    def assignWeights(self, method="Random"):
        """Takes a df and assigns weights (defaults to random, but can be either Random or Equal.)
        Returns a weights df with columns corresponding to the columns in the df arg.
        """
        # Dict to store weights as values to column name keys
        df_weights = {}

        # Since the df.columns length will at this point include % change values, weights length should be halved.
        weight_array_length = int(len(self.history_df.columns) / 2)
        
        weights = []
        for weight in range(weight_array_length):
            weights.append(random.randint(0, 10000))
            
        new_weights = []
        i = 0
        for weight in weights:
            new_weights.append(round(weights[i] / sum(weights), 2))
            i += 1
            
        # print("weights array:")
        # print(str(new_weights))
        
        # Check whether there are any errors and make the sum = 100
        if (sum(new_weights) < 1):
            # choose a random index
            randIndex = random.randint(0, len(new_weights) - 1)
            # print(new_weights[randIndex])
            new_weights[randIndex] += (1 - sum(new_weights))
            # print(new_weights)
        
        # Only assign random weights to the columns that don't contain %
        if (method == "Random"):
            # print("self.history_df.columns: "+str(self.history_df.columns))
            for item in self.history_df.columns:
                # print("\nitem: "+str(item))
                if "%" not in item:         
                    # print("% not in item.")
                    # Add to the dictionary
                    # print("df_weights[item] now being set to new_weights[self.history_df.get_loc(item)]: ")
                    # print(str(new_weights[self.history_df.columns.get_loc(item)]))
                    df_weights[str(item)] = new_weights[self.history_df.columns.get_loc(item)]
                
        elif (method == "Equal"):
            # Assign equally
            df[str(item)+" Weight"] = 1 / (len(df.columns) / 2)
            
        else:
            print("Choose weighting method: Random or Equal.")
            
        self.weights = df_weights
        # print(self.weights)
        return self.weights
        

    # Compute portfolio statustics
    def get_portfolio_stats(self):
        """Takes a portfolio price history df and its corresponding weights dict from an assign_weights function,
        and returns the weighted HPR and annualized return for the portfolio"""
        
        hpr_items = []
        stdev_items = []
        
        # Reset index to make using loc practical
        self.history_df = self.history_df.reset_index()
        non_date_columns = self.history_df.iloc[:, 1:]
        
        # Compute total cumulative % change for each asset
        for item in non_date_columns.columns:
            # print("\nitem: "+str(item))
            if ("%" not in item) & ("/" not in item):
                # print("% and / not in item.")
                
                beginning_price = non_date_columns.loc[0 , item]
                # print("Beginning price:")
                # print(beginning_price)
                
                ending_price = non_date_columns.loc[non_date_columns.index[-1], item]
                # print("Ending price:")
                # print(ending_price)
                
                item_hpr = (ending_price - beginning_price) / beginning_price
                # print("HPR for "+str(item)+":")
                # print(item_hpr)
                
                hpr_items.append(item_hpr)
                
            elif ("%" in item):
            # Compute standard deviation of daily % changes for each asset
                col_std = non_date_columns[item].std()
                stdev_items.append(col_std)
                
            
            # Multiply by weights...
            # First for HPR array
            weighted_hpr_items = []
            i = 0
            for item in hpr_items:
                weighted_hpr_items.append(hpr_items[i] * self.weights[non_date_columns.columns[i]])
                i += 1
            # Then for the stdev array
            weighted_stdev_items = []
            i = 0
            for item in stdev_items:
                weighted_stdev_items.append(stdev_items[i] * self.weights[non_date_columns.columns[i]])
                i += 1
                
        # print("weighted hpr_items: "+str(weighted_hpr_items))
        portfolio_hpr = sum(weighted_hpr_items)
        # print(portfolio_hpr)
        
        # print("weighted stdev_items: "+str(weighted_stdev_items))
        portfolio_stdev = sum(weighted_stdev_items)
        # print(portfolio_stdev)
        
        # Adjust by the period analyzed...
        
        # Compute the percentage of a trading year this period represents
        trading_days = 252
        df_days = len(non_date_columns.iloc[:, 0])
        # print("trading days in period analyzed: "+str(df_days))
        pct_of_year = df_days / trading_days
        # print("% of trading year: "+str(pct_of_year))
        
        # Extrapolate for annualized return using appropriate formula
        annualized_hpr = (1 + portfolio_hpr) ** (1 / pct_of_year) - 1
        # print("annualized return: "+str(annualized_hpr))
        
        # Apply a similar process to the standard deviation using the sqrt of period analyzed
        annualized_volatility = portfolio_stdev * np.sqrt(df_days)
        # print("annualized volatility: "+str(annualized_volatility))
        
        # Now compute annualized risk-adjusted return
        risk_adj_return = annualized_hpr / annualized_volatility
        # print("Risk-Adjusted Return: "+str(risk_adj_return))

        # Convert all of this information on the portfolio to its own dataframe
        portfolio_statistics = {
            "num_days": [df_days],
            "period_start": [self.history_df.loc[0, "Date"]],
            "period_end": [self.history_df.loc[self.history_df.index[-1], "Date"]],
            "num_stocks": [int(len(non_date_columns.columns) / 2)],
            "hpr": [portfolio_hpr],
            "annualized_return": [annualized_hpr],
            "stdev": [portfolio_stdev],
            "annualized_stdev": [annualized_volatility],
            "risk_adj_return": [risk_adj_return]
        }
        stats_df = pd.DataFrame(portfolio_statistics)
        self.analysis_df = stats_df
        
        # Output
        #   Write analysis metrics to analysis_df
        return self.analysis_df

    #   TODO: Compare analysis to benchmark for alpha
    def compare_benchmark(self, benchmark="SPY"):
        """Defaults to SPY, compares using the same n_periods and start_date params"""
        bench_df = pd.DataFrame({})   
        # print("Start date index: "+str(self.start_date_index))
        # print("n_periods: "+str(self.n_periods))
        # print("Start date: "+str(reset_df["Date"].iloc[self.start_date_index]))
        
        # Pull data for SPY
        # print("df.loc["+str(self.start_date_index)+" : "+str(self.start_date_index+self.n_periods)+", ["+str(benchmark)+"]]: ")
        bench_df[benchmark] = reset_df.loc[self.start_date_index : self.start_date_index + self.n_periods, [benchmark]]
        
        # Load with the percentage change columns
        for item in bench_df.columns:
    #       First, load with values
            bench_df[str(item)+" % chg"] = bench_df[item].pct_change()
            
        beginning_price = bench_df.loc[self.start_date_index , benchmark]
        # print("Beginning price:")
        # print(beginning_price)
                
        ending_price = bench_df.loc[bench_df.index[-1], benchmark]
        # print("Ending price:")
        # print(ending_price)
        
        bench_hpr = (ending_price - beginning_price) / beginning_price
        # print("HPR for "+str(benchmark)+":")
        # print(bench_hpr)
        
        # bench_performance_df["hpr"] = bench_hpr
        
        # Compute annualized return
        trading_days = 252
        df_days = len(bench_df.iloc[:, 0])
        # print("trading days in period analyzed: "+str(df_days))
        pct_of_year = df_days / trading_days
        # print("% of trading year: "+str(pct_of_year))
        
        # Extrapolate for annualized return using appropriate formula
        annualized_hpr = (1 + bench_hpr) ** (1 / pct_of_year) - 1
        # bench_performance_df["annualized_return"] = annualized_hpr
        # print("Annualized HPR: "+str(annualized_hpr))
        
        bench_stdev = bench_df[benchmark].std()
        # bench_performance_df["stdev"] = bench_stdev
        
        annualized_volatility = bench_stdev * np.sqrt(df_days)
        # bench_performance_df["annualized_stdev"] = annualized_volatility
        
        # bench_performance_df["risk_adj_return"] = annualized_hpr / annualized_volatility
        
        # Try a different approach to df creation
        bench_performance = {
            "hpr" : [bench_hpr],
            "annualized_return" : [annualized_hpr],
            "stdev" : [bench_stdev],
            "annualized_stdev" : [annualized_volatility],
            "risk_adj_return" : [annualized_hpr / annualized_volatility],
        }
        bench_performance_df = pd.DataFrame(bench_performance)
        
        # Compute metrics and return as a df
        # print("\nbench_performance_df.head(): ")
        # print(bench_performance_df.head(5))
        # print("\n-------------------------------\n")
        return bench_performance_df
        
    
    # Write included tickers and weights to a df
    def get_portfolio_comp(self):
        # return a df with columns corresponding to tickers and weights
        return pd.DataFrame(list(self.weights.items()), columns=["Ticker", "Weight"])


In [4]:
# Testbed
stats_df = pd.DataFrame({})
composition_df = pd.DataFrame({})
bench_df = pd.DataFrame({})
comp_df_cols = []
for x in range(10):
    
    portfolio = Portfolio(size=10, n_periods=30, start_date="10/19/2020")
    
    # Rename columns with a suffix based on the iteration
    suffix = "_"+str(x)
    comp_df_cols.append("Ticker"+suffix)
    comp_df_cols.append("Weight"+suffix)

    # Merge with outer join to preserve non-identical weights and tickers for each portfolio
    composition_df = pd.merge(composition_df, portfolio.get_portfolio_comp(), left_index=True, right_index=True, how="outer")
    stats_df = pd.concat([stats_df, portfolio.get_portfolio_stats()])

    # Store the benchmark comparison
    # bench_df = pd.merge(bench_df, portfolio.compare_benchmark(), left_index=True, right_index=True, how="inner") 
    bench_df = portfolio.compare_benchmark()
    
composition_df.columns = comp_df_cols
    
stats_df = stats_df.reset_index(drop=True)
    
stats_df.head(10)
# composition_df.head(100)

Initialized new Portfolio instance
Initialized new Portfolio instance
Initialized new Portfolio instance
Initialized new Portfolio instance
Initialized new Portfolio instance


C:\Users\Jonathan\AppData\Local\Temp\ipykernel_10044\2823040203.py:16: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Ticker_x', 'Weight_x'} in the result is deprecated and will raise a MergeError in a future version.
  composition_df = pd.merge(composition_df, portfolio.get_portfolio_comp(), left_index=True, right_index=True, how="outer")
C:\Users\Jonathan\AppData\Local\Temp\ipykernel_10044\2823040203.py:16: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Ticker_x', 'Weight_x'} in the result is deprecated and will raise a MergeError in a future version.
  composition_df = pd.merge(composition_df, portfolio.get_portfolio_comp(), left_index=True, right_index=True, how="outer")
C:\Users\Jonathan\AppData\Local\Temp\ipykernel_10044\2823040203.py:16: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Ticker_x', 'Weight_x'} in the result is deprecated and will raise a MergeError in a future version.
  composition_df = pd.merge(composition_d

Initialized new Portfolio instance
Initialized new Portfolio instance
Initialized new Portfolio instance
Initialized new Portfolio instance
Initialized new Portfolio instance


C:\Users\Jonathan\AppData\Local\Temp\ipykernel_10044\2823040203.py:16: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Ticker_x', 'Weight_x'} in the result is deprecated and will raise a MergeError in a future version.
  composition_df = pd.merge(composition_df, portfolio.get_portfolio_comp(), left_index=True, right_index=True, how="outer")


,num_days,period_start,period_end,num_stocks,hpr,annualized_return,stdev,annualized_stdev,risk_adj_return
0,30,10/19/2020,11/30/2020,10,0.107464,1.357047,0.034815,0.190691,7.116456
1,30,10/19/2020,11/30/2020,10,-0.015036,-0.119495,0.061221,0.335319,-0.356362
2,30,10/19/2020,11/30/2020,10,0.392289,15.118567,0.053631,0.293749,51.467637
3,30,10/19/2020,11/30/2020,10,0.017404,0.155967,0.039020,0.213722,0.729767
4,30,10/19/2020,11/30/2020,10,0.211262,4.002633,0.061325,0.335891,11.916467
5,30,10/19/2020,11/30/2020,10,0.279626,6.934108,0.058164,0.318579,21.765752
6,30,10/19/2020,11/30/2020,10,0.132414,1.842121,0.026339,0.144265,12.768970
7,30,10/19/2020,11/30/2020,10,0.115416,1.503059,0.030786,0.168623,8.913727
8,30,10/19/2020,11/30/2020,10,0.085767,0.996130,0.118937,0.651446,1.529106
9,30,10/19/2020,11/30/2020,10,0.096089,1.161247,0.017717,0.097041,11.966534


In [5]:
composition_df.head(30)

,Ticker_0,Weight_0,Ticker_1,Weight_1,Ticker_2,Weight_2,Ticker_3,Weight_3,Ticker_4,Weight_4,Ticker_5,Weight_5,Ticker_6,Weight_6,Ticker_7,Weight_7,Ticker_8,Weight_8,Ticker_9,Weight_9
0,PLBC,0.04,IT,0.17,HST,0.18,EPOR,0.07,EQC,0.13,PZZA,0.15,ARE,0.08,GTN,0.12,AGM-A,0.13,LMT,0.12
1,FMNB,0.14,AVID,0.08,SEBC,0.21,TORM,0.09,APGI,0.10,ODP,0.14,ASML,0.18,CLM,0.06,SMTC,0.18,ERIE,0.03
2,PBEV,0.00,SCCO,0.07,BEEN,0.01,ALE,0.14,LSCC,0.15,RYN,0.00,DHIL,0.19,HOFT,0.10,OTMN,0.17,LFUS,0.08
3,XRX,0.18,TIXC,0.10,MSEX,0.02,SYBT,0.22,PRPH,0.18,OIS,0.16,DYSL,0.05,SHOO,0.10,MCRI,0.06,CRH,0.08
4,OFIX,0.06,WWSG,0.04,L,0.03,PML,0.21,ANIP,0.02,ETN,0.09,MUE,0.02,MSVB,0.15,MQT,0.08,KSM,0.06
5,SWKS,0.06,VRSN,0.23,JNJ,0.21,NAZ,0.02,DOV,0.18,WELX,0.16,ELLO,0.04,ALB,0.06,TISI,0.01,PUK,0.15
6,EIX,0.01,RNVT,0.06,RVP,0.06,QDEL,0.15,OTMN,0.11,TTNP,0.14,IDT,0.05,AMRN,0.18,ELP,0.01,CTS,0.09
7,BLFS,0.16,WCC,0.05,XOM,0.03,CMCSA,0.08,SAM,0.10,NVG,0.01,FMX,0.16,WSM,0.10,CLHI,0.16,EQIX,0.08
8,FISI,0.18,ACFL,0.18,CAKE,0.18,VIV,0.02,RICK,0.01,CLDX,0.10,KLIC,0.15,RCMT,0.03,CAL,0.04,WVFC,0.16
9,CZBS,0.17,CLHI,0.02,NTZ,0.07,SFST,0.01,CNGA,0.02,INNX,0.05,CB,0.08,CATO,0.10,NAT,0.16,GILT,0.15


In [6]:
bench_df.head()

,hpr,annualized_return,stdev,annualized_stdev,risk_adj_return
0,0.070203,0.735925,11.195825,62.335714,0.011806


In [7]:
revised_stats_df = stats_df.dropna()
revised_stats_df = revised_stats_df.sort_values(by="risk_adj_return", ascending=False)
revised_stats_df.head(10)

,num_days,period_start,period_end,num_stocks,hpr,annualized_return,stdev,annualized_stdev,risk_adj_return
2,30,10/19/2020,11/30/2020,10,0.392289,15.118567,0.053631,0.293749,51.467637
5,30,10/19/2020,11/30/2020,10,0.279626,6.934108,0.058164,0.318579,21.765752
6,30,10/19/2020,11/30/2020,10,0.132414,1.842121,0.026339,0.144265,12.768970
9,30,10/19/2020,11/30/2020,10,0.096089,1.161247,0.017717,0.097041,11.966534
4,30,10/19/2020,11/30/2020,10,0.211262,4.002633,0.061325,0.335891,11.916467
7,30,10/19/2020,11/30/2020,10,0.115416,1.503059,0.030786,0.168623,8.913727
0,30,10/19/2020,11/30/2020,10,0.107464,1.357047,0.034815,0.190691,7.116456
8,30,10/19/2020,11/30/2020,10,0.085767,0.996130,0.118937,0.651446,1.529106
3,30,10/19/2020,11/30/2020,10,0.017404,0.155967,0.039020,0.213722,0.729767
1,30,10/19/2020,11/30/2020,10,-0.015036,-0.119495,0.061221,0.335319,-0.356362


In [8]:
#  See if there are any portfolios with better risk, return, or risk-adj return than the benchmark
alpha_portfolios_df = revised_stats_df[revised_stats_df["risk_adj_return"] >= bench_df.loc[0, "risk_adj_return"]]
return_portfolios_df = revised_stats_df[revised_stats_df["annualized_return"] >= bench_df.loc[0, "annualized_return"]]
risk_portfolios_df = revised_stats_df[revised_stats_df["annualized_stdev"] <= bench_df.loc[0, "annualized_stdev"]]

# Pick out the best from the list and explore its composition
best_portfolio = alpha_portfolios_df.iloc[0]

# alpha_portfolios_df["alpha"] = alpha_portfolios_df["risk_adj_return"] - bench_df["risk_adj_return"]
alpha_portfolios_df.head()


,num_days,period_start,period_end,num_stocks,hpr,annualized_return,stdev,annualized_stdev,risk_adj_return
2,30,10/19/2020,11/30/2020,10,0.392289,15.118567,0.053631,0.293749,51.467637
5,30,10/19/2020,11/30/2020,10,0.279626,6.934108,0.058164,0.318579,21.765752
6,30,10/19/2020,11/30/2020,10,0.132414,1.842121,0.026339,0.144265,12.768970
9,30,10/19/2020,11/30/2020,10,0.096089,1.161247,0.017717,0.097041,11.966534
4,30,10/19/2020,11/30/2020,10,0.211262,4.002633,0.061325,0.335891,11.916467


In [9]:
return_portfolios_df.head()

,num_days,period_start,period_end,num_stocks,hpr,annualized_return,stdev,annualized_stdev,risk_adj_return
2,30,10/19/2020,11/30/2020,10,0.392289,15.118567,0.053631,0.293749,51.467637
5,30,10/19/2020,11/30/2020,10,0.279626,6.934108,0.058164,0.318579,21.765752
6,30,10/19/2020,11/30/2020,10,0.132414,1.842121,0.026339,0.144265,12.768970
9,30,10/19/2020,11/30/2020,10,0.096089,1.161247,0.017717,0.097041,11.966534
4,30,10/19/2020,11/30/2020,10,0.211262,4.002633,0.061325,0.335891,11.916467


In [10]:
risk_portfolios_df.head()

,num_days,period_start,period_end,num_stocks,hpr,annualized_return,stdev,annualized_stdev,risk_adj_return
2,30,10/19/2020,11/30/2020,10,0.392289,15.118567,0.053631,0.293749,51.467637
5,30,10/19/2020,11/30/2020,10,0.279626,6.934108,0.058164,0.318579,21.765752
6,30,10/19/2020,11/30/2020,10,0.132414,1.842121,0.026339,0.144265,12.768970
9,30,10/19/2020,11/30/2020,10,0.096089,1.161247,0.017717,0.097041,11.966534
4,30,10/19/2020,11/30/2020,10,0.211262,4.002633,0.061325,0.335891,11.916467


In [11]:
# Explore the composition of the best portfolio
best_index = best_portfolio.name
composition_df.loc[:, ["Ticker_"+str(best_index), "Weight_"+str(best_index)]]

,Ticker_2,Weight_2
0,HST,0.18
1,SEBC,0.21
2,BEEN,0.01
3,MSEX,0.02
4,L,0.03
5,JNJ,0.21
6,RVP,0.06
7,XOM,0.03
8,CAKE,0.18
9,NTZ,0.07
